Fecha: Julio de 2025

Se busca hacer análisis de frecuencias en las columnas de opinión por año, mes y periódico. E implementar modelado de tópicos. Primer método LDA.

In [15]:
# Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [28]:
file_path = r'C:\Users\karen\Documents\HumanidadesDigitales_git\BBD_Corpus\ElEspectador_2018.csv'

try:
    df = pd.read_csv(file_path, encoding='latin-1', sep=';')  # Ajusta el separador si es necesario
    print(df.head())
except FileNotFoundError:
    print("Error: El archivo no fue encontrado.")
except Exception as e:
    print(f"Ocurrió un error: {e}")


          Diario                       Autor                    Fecha  \
0  El Espectador             Alberto Donadio  30 de diciembre de 2017   
1  El Espectador              Antonio Casale  30 de diciembre de 2017   
2  El Espectador           Gonzalo Hernández       1 de enero de 2018   
3  El Espectador    Eduardo Barajas Sandoval       1 de enero de 2018   
4  El Espectador  Daniel Emilio Rojas Castro       1 de enero de 2018   

                               Título  \
0     César Gaviria y César Mondragón   
1   Más recomendaciones de fin de año   
2            Fajardo: para nada tibio   
3                  Macedonia de Norte   
4  El nacionalismo según Vargas Llosa   

                                               Texto  \
0  César Gaviria y César Mondragón son expresiden...   
1  Un libro y dos documentales imperdibles para e...   
2  La Coalición Colombia Partido Alianza Verde, ...   
3  Las interpretaciones de la historia sirven com...   
4  La semana pasada Mario Vargas L

3634